In [1]:
""" mount drive to access the dataset"""

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
"""model.py"""

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

""" build_resnet50_unet is defined in'model' part of 'train.py' """

" build_resnet50_unet is defined in'model' part of 'train.py' "

In [3]:
"""metric.py"""

import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [4]:
"""train.py"""

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau # ModelCheckPoint - save weights;
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

""" Global parameters """
H = 512
W = 512

In [5]:
def create_dir(path):
    """ Create a directory. """
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(split=0.2): # split as 60-20-20 for train-val-test split
    images = sorted(glob(os.path.join( "/content/drive/MyDrive/Stanford-Computer_Vision-Rectina/Image_Dataset/Image", "*.jpg")))                       #
    masks = sorted(glob(os.path.join("/content/drive/MyDrive/Stanford-Computer_Vision-Rectina/Image_Dataset/Mask", "*.jpg")))        # EDIT ACCORDINGLY

    train_x, valid_x = train_test_split(images, test_size=split, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [6]:
def apply_clahe_img(image):
    # Convert RGB image to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

    # Apply CLAHE to the L channel (Lightness)
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    lab_planes[0] = clahe.apply(lab_planes[0])

    # Merge the enhanced L channel back with the other channels
    lab = cv2.merge(lab_planes)

    # Convert the LAB image back to RGB color space
    enhanced_image = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

    return enhanced_image

def apply_clahe_mask(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image = clahe.apply(image)
    return image

def read_image(path):       # read an image from train_x, or train_y, or... any other data split
    x = cv2.imread(path, cv2.IMREAD_COLOR)      # convert it to 3 channel (if grayscale or else colour only) and
    x = cv2.resize(x, (W, H))                   # resize image to (512,512,3)
    # x = apply_clahe_img(x)
    x = x/255.0                                 # normailise to have value bethween 0 and 1
    x = x.astype(np.float32)                    # convert to numpy float datatype
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    # x = apply_clahe_mask(x)
    x = x/np.max(x)
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)          # convert (512,512) to (512,512,1)
    return x

def tf_parse(x, y):        # this function takes in images x,y
    def _parse(x, y):      # calls read_image and read_mask on the image
        x = x.decode()
        y = y.decode()

        x = read_image(x)
        y = read_mask(y)
        return x, y             # returns as numpy array

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])     # converts numpy array to tensor type
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch=8):     # Here X,Y are list containing the images
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.shuffle(buffer_size=200)  # shuffle the dataset
    dataset = dataset.map(tf_parse)     # calls 'tf_parse' function to convert to a tensor
    dataset = dataset.batch(batch)      # create a batch of data
    dataset = dataset.prefetch(4)       # prefect some data in advance to RAM
    return dataset

In [7]:
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)          # to ensure that randomness is prevented, simmillar results are produced at later implementations
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 4      # to limit number of images eveluvated at once to handle GPU limitations
    lr = 1e-5           # learning rate
    num_epochs = 20     # number of iteations
    model_path = os.path.join("files", "model.h5")      # location where model weights from 'ModelCheckpoint' are stored
    csv_path = os.path.join("files", "data.csv")        # location where model csv details from 'CSVLogger' are stored

In [8]:
    """ Dataset """
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data()

In [9]:
    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

Train: 37 - 37
Valid: 12 - 12
Test: 10 - 10


In [10]:
    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)   # create train and test dataset out of x_train and y_train
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [11]:
    """ Model """

    def build_resnet50_unet(input_shape):
      """ Input """
      inputs = Input(input_shape)

      """ Pre-trained ResNet50 Model """
      resnet50 = ResNet50(include_top=False, weights="imagenet", input_tensor=inputs)
      # resnet50.summary()

      """ Encoder """
      s1 = resnet50.get_layer("input_1").output           ## (512 x 512)   -> use resnet50.summary() valuse to make changes incase of 'valueerror'
      s2 = resnet50.get_layer("conv1_relu").output        ## (256 x 256)
      s3 = resnet50.get_layer("conv2_block3_out").output  ## (128 x 128)
      s4 = resnet50.get_layer("conv3_block4_out").output  ## (64 x 64)

      """ Bridge """
      b1 = resnet50.get_layer("conv4_block6_out").output  ## (32 x 32)

      """ Decoder """
      d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
      d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
      d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
      d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)

      """ Output """
      outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

      model = Model(inputs, outputs, name="ResNet50_U-Net")
      return model

    model = build_resnet50_unet((H, W, 3))
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

    model.summary()


94765736/94765736 [==============================] - 0s 0us/step
Model: "ResNet50_U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                    

In [12]:
    # """Perform Data Augmentation on train and valid sets"""

    # import numpy as np
    # from keras.preprocessing.image import ImageDataGenerator
    # from skimage import io

    # datagen = ImageDataGenerator(
    #         rotation_range=45,     # Random rotation between 0 and 45
    #         width_shift_range=0.2,   # % shift
    #         height_shift_range=0.2,
    #         shear_range=0.2,
    #         zoom_range=0.2,
    #         horizontal_flip=True,
    #         fill_mode='nearest')

    # # Convert dataset objects to numpy arrays
    # train_images = np.array([train_x for train_x, _ in train_dataset])
    # train_labels = np.array([train_y for _, train_y in train_dataset])
    # valid_images = np.array([valid_x for valid_x, _ in valid_dataset])
    # valid_labels = np.array([valid_y for _, valid_y in valid_dataset])

    # # Generate augmented datasets using flow() method
    # augmented_train_dataset = datagen.flow(train_images, train_labels, batch_size=5)
    # augmented_valid_dataset = datagen.flow(valid_images, valid_labels, batch_size=5)


In [13]:
    callbacks = [                                                                  # defines callbacks
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path)
    ]

    model.fit(                                                                     # fits / trains data
            train_dataset,
            epochs=num_epochs,
            validation_data=valid_dataset,
            callbacks=callbacks
        )

Epoch 1/20
10/10 [==============================] - ETA: 0s - loss: 0.8643 - dice_coef: 0.1418 - iou: 0.0772 - recall: 0.7276 - precision: 0.0797
Epoch 1: val_loss improved from inf to 0.92913, saving model to files/model.h5
10/10 [==============================] - 103s 3s/step - loss: 0.8643 - dice_coef: 0.1418 - iou: 0.0772 - recall: 0.7276 - precision: 0.0797 - val_loss: 0.9291 - val_dice_coef: 0.0709 - val_iou: 0.0372 - val_recall: 0.4897 - val_precision: 0.0389 - lr: 1.0000e-05
Epoch 2/20
10/10 [==============================] - ETA: 0s - loss: 0.8504 - dice_coef: 0.1423 - iou: 0.0780 - recall: 0.8058 - precision: 0.0928
Epoch 2: val_loss improved from 0.92913 to 0.92753, saving model to files/model.h5
10/10 [==============================] - 10s 985ms/step - loss: 0.8504 - dice_coef: 0.1423 - iou: 0.0780 - recall: 0.8058 - precision: 0.0928 - val_loss: 0.9275 - val_dice_coef: 0.0725 - val_iou: 0.0377 - val_recall: 0.4936 - val_precision: 0.0381 - lr: 1.0000e-05
Epoch 3/20
10/10 [

In [14]:
"""eval.py"""

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope      # used to implement additional metrics such as iou, dice_coef, etc

""" Directory for storing files """
create_dir("results")

""" Loading model """# define functions that are externmal to tensorflow
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
    model = tf.keras.models.load_model("files/model.h5")

for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extracing the image name. """
        image_name = x.split("/")[-1]

        """ Reading the image """       # 'read_image' function
        ori_x = cv2.imread(x, cv2.IMREAD_COLOR)
        ori_x = cv2.resize(ori_x, (W, H))
        x = ori_x/255.0
        x = x.astype(np.float32)
        x = np.expand_dims(x, axis=0)

        """ Reading the mask """        # 'read_mask' function
        ori_y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
        ori_y = cv2.resize(ori_y, (W, H))
        ori_y = np.expand_dims(ori_y, axis=-1)  ## (512, 512, 1)
        ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  ## (512, 512, 3)

        """ Predicting the mask. """
        y_pred = model.predict(x)[0]> 0.5
        y_pred = y_pred.astype(np.int32)        # converting predicted result to integer datatype

        y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)

        """ Saving the predicted mask along with the image and GT """
        save_image_path = f"results/{image_name}"   # location to save image

        sep_line = np.ones((H, 10, 3)) * 255    # a white line with 10 pivel width

        cat_image = np.concatenate([ori_x, sep_line, ori_y, sep_line, y_pred*255], axis=1)  # original image | original mask | predicted mask [ori_x, sep_line, ori_y, sep_line, y_pred*255]

        cv2.imwrite(save_image_path, cat_image)

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 1s 941ms/step


 10%|█         | 1/10 [00:01<00:16,  1.84s/it]

1/1 [==============================] - 0s 29ms/step


 20%|██        | 2/10 [00:02<00:10,  1.32s/it]

...

 80%|████████  | 8/10 [00:08<00:01,  1.10it/s]

1/1 [==============================] - 0s 28ms/step


 90%|█████████ | 9/10 [00:09<00:00,  1.11it/s]

1/1 [==============================] - 0s 24ms/step


100%|██████████| 10/10 [00:09<00:00,  1.01it/s]
